<a href="https://colab.research.google.com/github/Chaitya0623/IPD_Medledger/blob/main/DiseasePred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
df = pd.read_csv('dataset1.csv')
disease_df = df[['name', 'symptoms']]
disease_df

,name,symptoms
0,Panic disorder,"{'Anxiety and nervousness': '88', 'Depression'..."
1,Vocal cord polyp,"{'Hoarse voice': '91', 'Sore throat': '47', 'D..."
2,Turner syndrome,"{'Groin mass': '27', 'Leg pain': '27', 'Hip pa..."
3,Cryptorchidism,"{'Symptoms of the scrotum and testes': '50', '..."
4,Poisoning due to ethylene glycol,"{'Abusing alcohol': '78', 'Fainting': '64', 'H..."
...,...,...
791,Kaposi sarcoma,"{'Leg pain': '45', 'Foot or toe pain': '45', '..."
792,Spondylolisthesis,"{'Low back pain': '80', 'Back pain': '79', 'Le..."
793,Pseudotumor cerebri,"{'Headache': '88', 'Diminished vision': '50', ..."
794,Conjunctivitis due to virus,"{'Eye redness': '91', 'Pain in eye': '66', 'It..."


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import ast
disease_df['symptoms'] = disease_df['symptoms'].apply(ast.literal_eval)
disease_df['symptoms_list'] = disease_df['symptoms'].apply(lambda x: list(x.keys()))
disease_df

,name,symptoms,symptoms_list
0,Panic disorder,"{'Anxiety and nervousness': '88', 'Depression'...","[Anxiety and nervousness, Depression, Shortnes..."
1,Vocal cord polyp,"{'Hoarse voice': '91', 'Sore throat': '47', 'D...","[Hoarse voice, Sore throat, Difficulty speakin..."
2,Turner syndrome,"{'Groin mass': '27', 'Leg pain': '27', 'Hip pa...","[Groin mass, Leg pain, Hip pain, Suprapubic pa..."
3,Cryptorchidism,"{'Symptoms of the scrotum and testes': '50', '...","[Symptoms of the scrotum and testes, Swelling ..."
4,Poisoning due to ethylene glycol,"{'Abusing alcohol': '78', 'Fainting': '64', 'H...","[Abusing alcohol, Fainting, Hostile behavior, ..."
...,...,...,...
791,Kaposi sarcoma,"{'Leg pain': '45', 'Foot or toe pain': '45', '...","[Leg pain, Foot or toe pain, Cough, Lip swelli..."
792,Spondylolisthesis,"{'Low back pain': '80', 'Back pain': '79', 'Le...","[Low back pain, Back pain, Leg pain, Hip pain,..."
793,Pseudotumor cerebri,"{'Headache': '88', 'Diminished vision': '50', ...","[Headache, Diminished vision, Dizziness, Pain ..."
794,Conjunctivitis due to virus,"{'Eye redness': '91', 'Pain in eye': '66', 'It...","[Eye redness, Pain in eye, Itchiness of eye, F..."


In [6]:
# Convert the list of symptoms to text
disease_df['symptoms_text'] = disease_df['symptoms_list'].apply(lambda x: ' '.join(x))

# Use TF-IDF for feature extraction
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
symptoms_tfidf = tfidf_vectorizer.fit_transform(disease_df['symptoms_text'])

In [9]:
disease_df['symptoms_list'][0]

['Anxiety and nervousness',
 'Depression',
 'Shortness of breath',
 'Depressive or psychotic symptoms',
 'Sharp chest pain',
 'Dizziness',
 'Insomnia',
 'Abnormal involuntary movements',
 'Chest tightness',
 'Palpitations',
 'Irregular heartbeat',
 'Breathing fast']

In [10]:
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

# Use K-means clustering as an example
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(symptoms_tfidf)

# Alternatively, use cosine similarity for pairwise similarity
similarity_matrix = cosine_similarity(symptoms_tfidf)

# Trying the data on a new set of symptoms
new_symptoms = []
while True:
  user_input = input("Enter the symptom. Write exit to finish: ")
  if user_input.lower() == 'exit':
    break
  new_symptoms.append(user_input)
new_symptoms = disease_df['symptoms_list'][0]
new_symptoms_text = ' '.join(new_symptoms)
new_symptoms_tfidf = tfidf_vectorizer.transform([new_symptoms_text])

# Predict the cluster for new symptoms using K-means
new_cluster = kmeans.predict(new_symptoms_tfidf)[0]

# Alternatively, find the most similar diseases based on cosine similarity
similarity_scores = similarity_matrix[:, new_symptoms_tfidf.indices[0]]

# Get the indices of the top 5 most similar diseases
top_5_indices = similarity_scores.argsort()[-5:][::-1]

# Get the names of the top 5 most similar diseases
top_5_diseases = df.loc[top_5_indices, 'name'].tolist()

print('Top 5 most similar diseases:')
for i, disease in enumerate(top_5_diseases, start=1):
    print(f'{i}. {disease}')

Enter the symptom. Write exit to finish: Insomnia
Enter the symptom. Write exit to finish: Anxiety and Nervousness
Enter the symptom. Write exit to finish: Dizziness
Enter the symptom. Write exit to finish: Depression
Enter the symptom. Write exit to finish: Shortness of breath
Enter the symptom. Write exit to finish: Chest tightness
Enter the symptom. Write exit to finish: palpitations
Enter the symptom. Write exit to finish: breathing fast
Enter the symptom. Write exit to finish: exit
Top 5 most similar diseases:
1. Gastroparesis
2. Gastritis
3. Indigestion
4. Intestinal disease
5. Hiatal hernia


In [11]:
from sklearn.neighbors import NearestNeighbors

# Assuming your DataFrame is named 'df'

# Extract the keys (symptoms) from the dictionary
disease_df['symptoms_list'] = disease_df['symptoms'].apply(lambda x: list(x.keys()))

# Convert the list of symptoms to text
disease_df['symptoms_text'] = disease_df['symptoms_list'].apply(lambda x: ' '.join(x))

# Use TF-IDF for feature extraction
tfidf_vectorizer = TfidfVectorizer()
symptoms_tfidf = tfidf_vectorizer.fit_transform(disease_df['symptoms_text'])

# Train a Nearest Neighbors model
nn_model = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='cosine')
nn_model.fit(symptoms_tfidf)

# Assuming 'new_symptoms' is a new set of symptoms
new_symptoms = []
while True:
  user_input = input("Enter the symptom. Write exit to finish: ")
  if user_input.lower() == 'exit':
    break
  new_symptoms.append(user_input)

# Convert the list of symptoms to text
new_symptoms_text = ' '.join(new_symptoms)

# Transform the new symptoms using the same TF-IDF vectorizer
new_symptoms_tfidf = tfidf_vectorizer.transform([new_symptoms_text])

# Find the most similar diseases using Nearest Neighbors
distances, indices = nn_model.kneighbors(new_symptoms_tfidf)

# Get the names of the most similar diseases
similar_diseases = disease_df.loc[indices[0][1:], 'name'].tolist()

print(f'Most similar diseases for new symptoms:')
for i, disease in enumerate(similar_diseases, start=1):
    print(f'{i}. {disease}')

Enter the symptom. Write exit to finish: Insomnia
Enter the symptom. Write exit to finish: Anxiety and Nervousness
Enter the symptom. Write exit to finish: Dizziness
Enter the symptom. Write exit to finish: Depression
Enter the symptom. Write exit to finish: Shortness of breath
Enter the symptom. Write exit to finish: Chest tightness
Enter the symptom. Write exit to finish: palpitations
Enter the symptom. Write exit to finish: breath fast
Enter the symptom. Write exit to finish: exit
Most similar diseases for new symptoms:
1. Panic attack
2. Acute stress reaction
3. Anxiety
4. Poisoning due to antipsychotics
5. Hypertensive heart disease
